### Zero-Shot Prompting:
 You provide the instruction and the input, and the LLM generates the output without any examples. This is the simplest approach.<br>
### Few Shot Prompting:
Few-Shot Prompting: You provide a few examples of input-output pairs along with the instruction. This helps the LLM understand the desired format, style, or task, especially for tasks that are not immediately obvious or require a specific pattern.<br>
**FewShotPromptTemplate(), FewShotChatMessagePromptTemplate()**

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import dotenv
dotenv.load_dotenv()

google_apii_key = os.getenv("GOOGLE_API_KEY")

llm= ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=google_apii_key,
)

In [6]:
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate, AIMessagePromptTemplate, SystemMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessagePromptTemplate.from_template("{output}")
])
        
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=[
        {
            "input": "What is the capital of France?",
            "output": "The capital of France is Paris."
        },
        {
            "input": "What is the largest planet in our solar system?",
            "output": "The largest planet in our solar system is Jupiter."
        },
    ],
    input_variables=["input"],
)

final_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant use the given exmaple for reference."),
    few_shot_prompt,
    HumanMessagePromptTemplate.from_template("{input}"),
])
input_text = "What is the tallest mountain in the world?"
response = final_prompt.format_messages(input=input_text)
for message in response:
    print(message.content)
    
llm_response= llm.invoke(response)
print("LLM Response:", llm_response.content)


You are a helpful assistant use the given exmaple for reference.
What is the capital of France?
The capital of France is Paris.
What is the largest planet in our solar system?
The largest planet in our solar system is Jupiter.
What is the tallest mountain in the world?
LLM Response: The tallest mountain in the world, measured from base to peak, is Mount Everest.


## Chain of Thoughts CoT <br>
Chain of Thought (CoT) Prompting: You instruct the LLM to show its reasoning steps before providing the final answer. This helps the LLM break down complex problems, improves accuracy on reasoning tasks, and makes the LLM's thought process transparent.


In [10]:
prompt_without_CoT = ChatPromptTemplate.from_messages([
    ("human", "Question: If a baker bakes 20 cookies and sells 12, then bakes 10 more, how many cookies does the baker have now?")
    ])
prompt_with_CoT = ChatPromptTemplate.from_messages([
    ("human", "Question: If a baker bakes 20 cookies and sells 12, then bakes 10 more, how many cookies does the baker have now?\n\nLet's think step by step.")
    ])
prompt_complex_cot = ChatPromptTemplate.from_messages([
        ("human", "Question: A is older than B. B is older than C. C is older than D. Who is the youngest?\n\nLet's break this down step by step to find the answer.")
    ])

response_without_CoT = llm.invoke(prompt_without_CoT.format_messages())
print("Response without CoT:", response_without_CoT.content,"\n\n")

response_with_CoT = llm.invoke(prompt_with_CoT.format_messages())
print("Response with CoT:", response_with_CoT.content,"\n\n")

response_complex_cot = llm.invoke(prompt_complex_cot.format_messages())
print("Response with complex CoT:", response_complex_cot.content)



Response without CoT: Here's how to solve the problem:

*   **Start:** The baker has 20 cookies.
*   **Sells:** The baker sells 12 cookies, leaving 20 - 12 = 8 cookies.
*   **Bakes more:** The baker bakes 10 more cookies, so they have 8 + 10 = 18 cookies.

**Answer:** The baker has 18 cookies now. 


Response with CoT: Okay, let's break it down:

1. **Starts with:** The baker begins with 20 cookies.
2. **Sells some:** The baker sells 12 cookies, so 20 - 12 = 8 cookies are left.
3. **Bakes more:** The baker bakes 10 more cookies, so 8 + 10 = 18 cookies.

**Answer:** The baker has 18 cookies now. 


Response with complex CoT: Okay, let's break it down:

*   **A is older than B:** This means B is younger than A.
*   **B is older than C:** This means C is younger than B.
*   **C is older than D:** This means D is younger than C.

So we have A > B > C > D (where ">" means "is older than").

Therefore, **D** is the youngest.


## Tree of Thoughts ToT
An extension of CoT, ToT allows the LLM to explore multiple reasoning paths (a "tree" of thoughts) and self-evaluate them, pruning less promising paths. This is even more powerful for highly complex problems but also more computationally intensive.

It typically requires an agentic loop, where the LLM generates multiple "thought candidates," an evaluation mechanism (which might be another LLM call or a heuristic), and a selection process to decide which path to pursue. It often involves a search algorithm (like breadth-first search or depth-first search) over the thought tree.